
# Trail Corridor → Tiles Notebook (Trailbird)

This notebook helps you:
1. Load a trail (PCT/CDT/AT segment) from GeoJSON/GPX.
2. Build a *buffer* (aka **hose**) of N meters around the trail.
3. Visualize trail + buffer on an interactive map.
4. Compute which XYZ tiles (for chosen zoom levels) intersect the buffer.
5. **Optionally** download tiles and bundle them into an **MBTiles** file for offline use.

> ⚠️ **Licensing & Terms**: Before downloading any tiles, verify the provider's Terms of Service and the license (OSM, USGS, OpenMapTiles, MapTiler, etc). Many sources require attribution and may restrict bulk/offline downloads or commercial redistribution.



## 0) Setup
Run this cell once on your machine to install requirements.


In [ ]:

# If running locally, uncomment the following and run once.
# %pip install geopandas shapely folium mercantile pyproj tqdm requests sqlite-utils gpxpy



## 1) Imports & helpers


In [20]:

import json
from pathlib import Path

import geopandas as gpd
from shapely.geometry import LineString, shape, mapping, Polygon
from shapely.ops import transform as shp_transform
from pyproj import Transformer, CRS
import folium
import mercantile
from tqdm.auto import tqdm
import sqlite3
import math
import requests  # used only if you choose to download tiles

# ---- Projection helpers ----
# We'll project to Web Mercator (EPSG:3857) to buffer in meters, then project back to WGS84 (EPSG:4326).
crs_wgs84 = CRS.from_epsg(4326)
crs_merc = CRS.from_epsg(3857)
to_merc = Transformer.from_crs(crs_wgs84, crs_merc, always_xy=True).transform
to_wgs = Transformer.from_crs(crs_merc, crs_wgs84, always_xy=True).transform

def buffer_line_in_meters(line_geom, buffer_m):
    # Project to mercator, buffer in meters, project back to WGS84
    line_merc = shp_transform(to_merc, line_geom)
    buf_merc = line_merc.buffer(buffer_m)
    buf_wgs = shp_transform(to_wgs, buf_merc)
    return buf_wgs

def load_trail_geojson(path: str):
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"No such file: {path}")
    with open(p, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # Accept LineString or Feature(Geometry)
    if data.get('type') == 'Feature':
        geom = shape(data['geometry'])
    elif data.get('type') in ('LineString', 'MultiLineString'):
        geom = shape(data)
    elif data.get('type') == 'FeatureCollection':
        # merge first lines (simple case)
        for feat in data['features']:
            geom = shape(feat['geometry'])
            if geom.type in ('LineString', 'MultiLineString'):
                return geom
        raise ValueError('No LineString found in FeatureCollection')
    else:
        raise ValueError('Unsupported GeoJSON format')
    return geom

def tiles_for_polygon(polygon: Polygon, zmin: int, zmax: int):
    """Return a dict zoom->set( (x,y) ) for Web Mercator XYZ tiles intersecting polygon bbox, filtered by intersection test."""
    if polygon.is_empty:
        return {}

    # Work entirely in WGS84 lon/lat. mercantile gives tile bounds in lon/lat.
    minx, miny, maxx, maxy = polygon.bounds
    results = {}
    for z in range(zmin, zmax+1):
        tiles = set()
        # Iterate tiles covering the bbox
        for tile in mercantile.tiles(minx, miny, maxx, maxy, [z]):
            # Build tile polygon in lon/lat
            bbox = mercantile.bounds(tile)
            tile_poly = Polygon([
                (bbox.west, bbox.south),
                (bbox.east, bbox.south),
                (bbox.east, bbox.north),
                (bbox.west, bbox.north)
            ])
            if tile_poly.intersects(polygon):
                tiles.add((tile.x, tile.y))
        results[z] = tiles
    return results

def estimate_raster_size(num_tiles: int, avg_kb_per_tile: int = 20):
    # Rough estimate for PNG raster tiles (~10–50 KB each at mid zooms)
    return num_tiles * avg_kb_per_tile / 1024

def xyz_to_tms_y(y: int, z: int) -> int:
    # MBTiles "TMS" scheme requires flipping the y
    return (2**z - 1) - y

def init_mbtiles(path_mbtiles: str, metadata: dict):
    conn = sqlite3.connect(path_mbtiles)
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS tiles (zoom_level INTEGER, tile_column INTEGER, tile_row INTEGER, tile_data BLOB)")
    cur.execute("CREATE TABLE IF NOT EXISTS metadata (name TEXT, value TEXT)")
    cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS tile_index ON tiles (zoom_level, tile_column, tile_row)")
    for k, v in metadata.items():
        cur.execute("INSERT INTO metadata(name, value) VALUES (?, ?)", (k, str(v)))
    conn.commit()
    return conn

def download_tile(url_template: str, z: int, x: int, y: int, session: requests.Session):
    url = url_template.format(z=z, x=x, y=y)
    resp = session.get(url, timeout=20)
    resp.raise_for_status()
    return resp.content

def pack_tiles_to_mbtiles(url_template: str, tiles_index: dict, out_path: str, metadata: dict, max_tiles: int = None):
    """Download tiles from url_template and write to MBTiles. url_template e.g. https://tile.openstreetmap.org/{z}/{x}/{y}.png
    NOTE: Respect provider ToS and attribution. For dev/testing only unless licensed for bulk/offline use."""
    conn = init_mbtiles(out_path, metadata)
    cur = conn.cursor()
    count = 0
    with requests.Session() as s:
        for z, xyset in tiles_index.items():
            for x, y in tqdm(xyset, desc=f"z{z}"):
                if max_tiles and count >= max_tiles:
                    break
                try:
                    data = download_tile(url_template, z, x, y, s)
                    tms_y = xyz_to_tms_y(y, z)
                    cur.execute("INSERT OR REPLACE INTO tiles(zoom_level, tile_column, tile_row, tile_data) VALUES (?,?,?,?)",
                                (z, x, tms_y, data))
                    count += 1
                except Exception as e:
                    print(f"Failed {z}/{x}/{y}: {e}")
            if max_tiles and count >= max_tiles:
                break
    conn.commit()
    conn.close()
    print(f"Wrote {count} tiles to {out_path}")



## 2) Load a trail (GeoJSON/GPX) and build a buffer
- Provide a path to your **LineString** GeoJSON (or GPX) for the PCT/CDT/AT segment.  
- Choose a **buffer width** in meters (e.g., 3000–6000 m).


In [21]:

# Option A: Load from your file (uncomment and set path)
# trail_geom = load_trail_geojson('data/my_trail_segment.geojson')

# Option B: Tiny demo LineString (replace with real trail)
coords_demo = [
    (-121.505, 38.577),  # near Sacramento (just demo coords)
    (-120.0, 39.0),
    (-119.5, 39.5),
]
trail_geom = LineString(coords_demo)

buffer_meters = 5000  # 5 km
corridor_poly = buffer_line_in_meters(trail_geom, buffer_meters)

print(f"Trail length (deg-based): ~{trail_geom.length:.4f} (this is not km; for display only)")


Trail length (deg-based): ~2.2704 (this is not km; for display only)



## 3) Visualize trail + buffer on an interactive map


In [23]:

# Create a Folium map
# Center on the first coordinate
center_latlon = (trail_geom.coords[0][1], trail_geom.coords[0][0])
m = folium.Map(location=center_latlon, zoom_start=8, tiles='OpenStreetMap')

# Add trail line
folium.GeoJson(mapping(trail_geom), name='Trail', style_function=lambda x: {'color': 'blue', 'weight': 3}).add_to(m)

# Add buffer polygon
folium.GeoJson(mapping(corridor_poly), name='Corridor', style_function=lambda x: {'color': 'red', 'weight': 1, 'fillOpacity': 0.2}).add_to(m)

folium.LayerControl().add_to(m)
m



## 4) Compute intersecting tiles for a zoom range
Pick sensible zooms for hiking (e.g., **z10–z16**). Higher zoom = more tiles = bigger offline size.


In [24]:

zoom_min, zoom_max = 10, 14   # start modest; increase later
tiles_idx = tiles_for_polygon(corridor_poly, zoom_min, zoom_max)

counts = {z: len(xy) for z, xy in tiles_idx.items()}
total = sum(counts.values())
print("Tile counts per zoom:", counts)
print("Total tiles:", total)
print("Rough raster size estimate (~20KB/tile): ~{:.1f} MB".format(estimate_raster_size(total)))


Tile counts per zoom: {10: 13, 11: 28, 12: 67, 13: 193, 14: 616}
Total tiles: 917
Rough raster size estimate (~20KB/tile): ~17.9 MB



## 5) (Optional) Download tiles and build an **MBTiles** file

> ⚠️ **Respect Terms**: Bulk/offline tile download is often restricted. Use only providers that **explicitly allow** it, or tiles you **generate/self-host**. Provide attribution (e.g., “© OpenStreetMap contributors”).

- Example XYZ template for OSM (for illustration only): `https://tile.openstreetmap.org/{z}/{x}/{y}.png` — **not** allowed for bulk/offline.
- For allowed sources, swap in your licensed/offline-friendly URL (e.g., your own tile server).


In [ ]:

# Example usage (disabled by default). Replace `url_template` with your own tile server that permits offline use.
# url_template = "https://YOUR_TILE_SERVER/{z}/{x}/{y}.png"  # or .pbf for vector, handled differently
# out_mbtiles = "offline_pack.mbtiles"
# meta = {
#     'name': 'Trail corridor offline pack',
#     'format': 'png',
#     'attribution': '© OpenStreetMap contributors',
#     'type': 'baselayer',
#     'description': f'Buffer {buffer_meters}m, z{zoom_min}-{zoom_max}'
# }
# pack_tiles_to_mbtiles(url_template, tiles_idx, out_mbtiles, meta, max_tiles=None)
# print("Done.")



## 6) Vector tiles option (recommended for style & size)
For **vector tiles** (PBF), the MBTiles schema differs (you store raw PBFs and set `format=pbf` and `metadata['json']` for the style/layers).  
For production:
- Generate corridor-specific vector tiles from OSM using **Tilemaker** or **OpenMapTiles** toolchain.
- Or pre-cut your own raster **hillshade/contours** from USGS DEMs and include as overlays.

## 7) Attribution & licensing checklist
- ✅ Verify provider ToS allows offline/bulk use.
- ✅ Include required **attribution** text in your app.
- ✅ Store source + date + licence in your pack metadata for transparency.



## 5b) (Vector) Download **MVT (.pbf)** tiles into an XYZ directory

> Use this when your provider explicitly allows offline/bulk use (or when you self-host).  
> This stores **`{z}/{x}/{y}.pbf`** files you can serve via `file://` URLs on-device or pack into MBTiles.

**How to use with Mapbox/MapLibre on mobile (React Native):**
- **Directory mode** (simplest): copy the `tiles/` folder to app storage and set your vector source with `file:///.../tiles/{z}/{x}/{y}.pbf`.
- **MBTiles mode**: pack into an `.mbtiles` (SQLite). On mobile, you’ll need a small bridge/fetcher to read from MBTiles and feed bytes (since rnmapbox doesn’t natively read MBTiles). Directory mode is easier.

> ⚠️ **USGS vector tiles**: Many are served via **Esri Vector Tile Services**. Data ≈ public domain, but service ToS may disallow bulk scraping. Prefer **self-hosted vector tiles** generated from USGS/OSM data, or obtain explicit permission.


In [25]:

from pathlib import Path
from tqdm.auto import tqdm
import requests
import os

def download_vector_tiles(url_template:str, tiles_index:dict, out_dir:str, max_tiles:int=None):
    """Download MVT (.pbf) tiles into an XYZ folder structure.
    url_template example: 'https://YOUR_VECTOR_TILES/{z}/{x}/{y}.pbf'
    """
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)
    count = 0
    with requests.Session() as s:
        for z, xyset in tiles_index.items():
            for x, y in tqdm(xyset, desc=f"z{z}"):
                if max_tiles and count >= max_tiles:
                    break
                url = url_template.format(z=z, x=x, y=y)
                try:
                    r = s.get(url, timeout=20)
                    r.raise_for_status()
                    tile_path = out / str(z) / str(x)
                    tile_path.mkdir(parents=True, exist_ok=True)
                    with open(tile_path / f"{y}.pbf", 'wb') as f:
                        f.write(r.content)
                    count += 1
                except Exception as e:
                    print(f"Failed {url}: {e}")
            if max_tiles and count >= max_tiles:
                break
    print(f"Downloaded {count} tiles into {out_dir}")

# EXAMPLE (disabled): replace with your permitted vector tiles endpoint
# vec_url = 'https://YOUR_VECTOR_TILES/{z}/{x}/{y}.pbf'
# download_vector_tiles(vec_url, tiles_idx, out_dir='tiles_vector_xyz', max_tiles=5000)



### Optional: Pack vector tiles into **MBTiles** (`format=pbf`)

This writes the same XYZ set into an MBTiles file usable by servers and some viewers.  
Mobile SDKs typically need a small adapter to read MBTiles → bytes; directory mode is simpler for React Native.


In [26]:

import sqlite3, gzip

def init_mbtiles_vector(path_mbtiles:str, name:str, attribution:str, description:str, bounds=None, center=None, minzoom=None, maxzoom=None):
    conn = sqlite3.connect(path_mbtiles)
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS tiles (zoom_level INTEGER, tile_column INTEGER, tile_row INTEGER, tile_data BLOB)")
    cur.execute("CREATE TABLE IF NOT EXISTS metadata (name TEXT, value TEXT)")
    cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS tile_index ON tiles (zoom_level, tile_column, tile_row)")
    meta = {
        'name': name,
        'format': 'pbf',
        'attribution': attribution,
        'type': 'baselayer',
        'description': description,
        'version': '1'
    }
    if bounds: meta['bounds'] = ','.join(map(str,bounds))
    if center: meta['center'] = ','.join(map(str,center))
    if minzoom is not None: meta['minzoom'] = str(minzoom)
    if maxzoom is not None: meta['maxzoom'] = str(maxzoom)
    for k,v in meta.items():
        cur.execute("INSERT INTO metadata(name,value) VALUES(?,?)", (k, v))
    conn.commit()
    return conn

def pack_vector_xyz_to_mbtiles(xyz_root:str, tiles_index:dict, out_mbtiles:str, name='Trail corridor vector', attribution='Data sources vary', description='Vector tiles pack', gzip_tiles=True):
    conn = init_mbtiles_vector(out_mbtiles, name, attribution, description)
    cur = conn.cursor()
    count = 0
    for z, xyset in tiles_index.items():
        for x, y in tqdm(xyset, desc=f"z{z}"):
            path = Path(xyz_root) / str(z) / str(x) / f"{y}.pbf"
            if not path.exists():
                continue
            with open(path, 'rb') as f:
                data = f.read()
            if gzip_tiles:
                data = gzip.compress(data)
            tms_y = xyz_to_tms_y(y, z)
            cur.execute("INSERT OR REPLACE INTO tiles(zoom_level, tile_column, tile_row, tile_data) VALUES (?,?,?,?)", (z, x, tms_y, data))
            count += 1
    conn.commit()
    conn.close()
    print(f"Packed {count} tiles into {out_mbtiles}")

# EXAMPLE (disabled):
# pack_vector_xyz_to_mbtiles('tiles_vector_xyz', tiles_idx, out_mbtiles='vector_pack.mbtiles',
#                            name='Trail corridor (vector)', attribution='© contributors', description='Corridor z10-14')



### Create a **local style.json** that points to `file://` tiles

MapLibre GL (and Mapbox GL with appropriate permissions) can load a vector source from a `file://` URL template.
On mobile, you'll replace the path with the app's document directory path.

> Note: iOS/Android paths differ; in React Native, use `react-native-fs` or native APIs to get the absolute path, then set `tiles: [\`file:///ABSOLUTE/PATH/tiles_vector_xyz/{z}/{x}/{y}.pbf\`]`.


In [28]:

import json, os
from pathlib import Path

def write_local_style_json(xyz_root:str, out_path='local_style.json'):
    abs_root = Path(xyz_root).resolve().as_posix()
    style = {
      "version": 8,
      "name": "Local Corridor Vector",
      "sources": {
        "corridor": {
          "type": "vector",
          "tiles": [f"file://{abs_root}/{{z}}/{{x}}/{{y}}.pbf"],
          "minzoom": 0,
          "maxzoom": 22,
          "scheme": "xyz"
        }
      },
      "layers": [
        {"id": "land", "type": "fill", "source": "corridor", "source-layer": "landuse", "paint": {"fill-color": "#e8e8e8"}},
        {"id": "roads", "type": "line", "source": "corridor", "source-layer": "transportation", "paint": {"line-color": "#888", "line-width": 1}},
        {"id": "water", "type": "fill", "source": "corridor", "source-layer": "water", "paint": {"fill-color": "#a0c8f0"}}
      ]
    }
    with open(out_path, 'w', encoding='utf-8') as f:
        json.dump(style, f, indent=2)
    print('Wrote', out_path)

# EXAMPLE (disabled)
# write_local_style_json('tiles_vector_xyz', out_path='local_style.json')



## 8) **USGS‑Allowed Path**: Build offline tiles from public‑domain source data

You will **not** scrape basemap tiles. Instead, you’ll download **source datasets** from **USGS The National Map (TNM)**, then produce your own corridor tiles.

### Overview
1. Use the **corridor polygon** from earlier.
2. Use **TNM Access API** to find **3DEP DEM** tiles for that area.
3. Download DEM tiles → mosaic → (optional) clip to corridor.
4. Derive **hillshade** and **contours** (vector).
5. Convert contours (and any other vectors) to **MVT** with **tippecanoe** (vector tiles).
6. (Optional) Generate **OSM base vector tiles** for the same corridor using **Tilemaker** (pure OSM, permissive).
7. Reference these local tiles from MapLibre/Mapbox in the app.

> **Docs:** TNM API: https://tnmaccess.nationalmap.gov/api/v1/products



### 8.1 Query TNM for 3DEP DEM covering your corridor

We’ll compute a bounding box from the corridor and query TNM for **3DEP** elevation products (e.g., 1 arc‑second). You can change `datasets` / `prodFormats` to suit.


In [29]:

import requests, json
from shapely.geometry import shape, mapping, Polygon

TNM_BASE = "https://tnmaccess.nationalmap.gov/api/v1/products"


# dataset names TNM accepts, e.g.:
# "Digital Elevation Model (DEM) 1 meter"
# "Digital Elevation Model (DEM) 1/9 arc-second"
# "Digital Elevation Model (DEM) 1/3 arc-second"
# "Digital Elevation Model (DEM) 1 arc-second"

def tnm_search_dem_bbox(corridor_poly,
                        dataset_label="Digital Elevation Model (DEM) 1/3 arc-second",
                        prod_format="GeoTIFF",
                        max_items=200, offset=0):
    minx, miny, maxx, maxy = corridor_poly.bounds  # lon/lat
    params = {
        "datasets": dataset_label,
        "prodFormats": prod_format,
        "bbox": f"{miny},{minx},{maxy},{maxx}",   # south,west,north,east
        "outputFormat": "JSON",
        "max": str(max_items),
        "offset": str(offset),
        # IMPORTANT: do NOT send polyType/polyCode when using bbox
        # "polyType": "", "polyCode": ""
    }
    r = requests.get(TNM_BASE, params=params, timeout=60)  # TLS verify=True by default
    r.raise_for_status()
    try:
        return r.json()
    except Exception:
        print("Status:", r.status_code)
        print("First 400 chars:", r.text[:400])
        raise

# Example (disabled by default if no internet here):
dem_listing = tnm_search_dem_bbox(corridor_poly)
print(dem_listing.keys(), dem_listing.get('total'), 'items')
dem_urls = [it.get('downloadURL') for it in dem_listing.get('items', []) if it.get('downloadURL')]
dem_urls[:5]


dict_keys(['total', 'items', 'errors', 'messages', 'sciencebaseQuery', 'filteredOut']) 0 items


[]


### 8.2 Download & mosaic DEM, then derive hillshade & contours

These steps use **GDAL** (and optionally Rasterio). Install GDAL from your package manager or conda; tippecanoe from its releases.

```bash
# (One-time) install via Homebrew on macOS:
brew install gdal tippecanoe

# On Linux, use apt or your distro package manager
sudo apt-get install -y gdal-bin tippecanoe
```


In [ ]:

# 8.2.1 Download DEM tiles (uncomment when running locally)
# from pathlib import Path
# import requests
# dem_dir = Path('usgs_dem')
# dem_dir.mkdir(exist_ok=True)
# for url in dem_urls:
#     fn = dem_dir / url.split('/')[-1]
#     if fn.exists(): 
#         continue
#     print('Downloading', url)
#     with requests.get(url, stream=True) as r:
#         r.raise_for_status()
#         with open(fn, 'wb') as f:
#             for chunk in r.iter_content(chunk_size=1<<20):
#                 if chunk:
#                     f.write(chunk)
# print('DEM files in', dem_dir)

# 8.2.2 Mosaic DEM to one GeoTIFF (uses gdal)
# !gdalbuildvrt dem.vrt usgs_dem/*.tif
# !gdal_translate -of COG dem.vrt dem_mosaic.tif

# 8.2.3 Clip to corridor bbox (optional; for speed). Use corridor bounds computed earlier.
# minx, miny, maxx, maxy = corridor_poly.bounds
# !gdalwarp -te {minx} {miny} {maxx} {maxy} -te_srs EPSG:4326 dem_mosaic.tif dem_clip.tif

# 8.2.4 Generate hillshade (nice for map background)
# !gdaldem hillshade dem_clip.tif hillshade.tif -compute_edges -multidirectional

# 8.2.5 Generate contours every 10m (adjust interval)
# !gdal_contour -a elev -i 10 dem_clip.tif contours_10m.geojson



### 8.3 Build **vector tiles (MVT)** for contours (and other layers)

We’ll use **tippecanoe** to produce an **MBTiles** (or a directory) of vector tiles for contours. You can later add other layers (hydrography, closures, etc.).


In [ ]:

# MBTiles (vector) for contours; adjust min/max zooms to your needs
# !tippecanoe -o contours.mbtiles -zg --drop-densest-as-needed \
#   --layer=contours contours_10m.geojson

# Or write an XYZ directory instead of MBTiles:
# !tippecanoe -e tiles_contours_xyz -zg --drop-densest-as-needed \
#   --layer=contours contours_10m.geojson



### 8.4 Optional: OSM base vector tiles for the corridor (self-hosted)

For general basemap (roads, landuse, POIs), build **OSM vector tiles** yourself:
- **Tilemaker** (simple pipeline): https://tilemaker.org  
- **OpenMapTiles** (full schema): https://openmaptiles.org

**Tilemaker example (corridor‑bounded extract):**
```bash
# 1) Download a regional PBF (e.g., geofabrik for your states)
wget https://download.geofabrik.de/north-america/us/california-latest.osm.pbf -O region.osm.pbf

# 2) Clip to corridor bbox (osmium or osmium-tool)
osmium extract -b <minlon,minlat,maxlon,maxlat> region.osm.pbf -o corridor.osm.pbf

# 3) Generate vector tiles (XYZ directory)
tilemaker --input corridor.osm.pbf --output tiles_osm_xyz --config resources/config-openmaptiles.json --process resources/process-openmaptiles.lua
```

Then you can reference both **OSM tiles** and your **contours tiles** in the map style.



### 8.5 Compose a local **style.json** with multiple vector sources

This creates a style that references your **OSM base** (`tiles_osm_xyz`) and **contours** (`tiles_contours_xyz`) using `file://` URLs.


In [ ]:

import json
from pathlib import Path

def write_multi_source_style(osm_xyz:str, contours_xyz:str, out_path='local_style_multi.json'):
    osm_abs = Path(osm_xyz).resolve().as_posix()
    con_abs = Path(contours_xyz).resolve().as_posix()
    style = {
      "version": 8,
      "name": "Corridor Offline (OSM + Contours)",
      "sources": {
        "osm": {
          "type": "vector",
          "tiles": [f"file://{osm_abs}/{{z}}/{{x}}/{{y}}.pbf"],
          "minzoom": 0, "maxzoom": 22, "scheme": "xyz"
        },
        "contours": {
          "type": "vector",
          "tiles": [f"file://{con_abs}/{{z}}/{{x}}/{{y}}.pbf"],
          "minzoom": 0, "maxzoom": 22, "scheme": "xyz"
        }
      },
      "layers": [
        {"id":"landuse","type":"fill","source":"osm","source-layer":"landuse","paint":{"fill-color":"#eef2ea"}},
        {"id":"water","type":"fill","source":"osm","source-layer":"water","paint":{"fill-color":"#a0c8f0"}},
        {"id":"roads","type":"line","source":"osm","source-layer":"transportation","paint":{"line-color":"#666","line-width":1}},
        {"id":"contours","type":"line","source":"contours","source-layer":"contours","paint":{"line-color":"#8b6f47","line-width":0.5}}
      ]
    }
    with open(out_path, 'w', encoding='utf-8') as f:
        json.dump(style, f, indent=2)
    print('Wrote', out_path)

# EXAMPLE (disabled):
# write_multi_source_style('tiles_osm_xyz', 'tiles_contours_xyz', out_path='local_style_multi.json')



### 8.6 Using in React Native (MapLibre/Mapbox)

- Copy `tiles_osm_xyz/` and `tiles_contours_xyz/` into app storage (e.g., Documents dir).
- Copy `local_style_multi.json` alongside them and **update the `file://` paths** at runtime to actual device paths.
- Set the style on your map view to the **local style**.

> If you need MBTiles instead of XYZ directories, use tippecanoe’s `-o *.mbtiles` options and implement a small on-device tile fetcher to read from MBTiles and serve bytes to the map SDK.
